Source: CustomGPT AI Automation Academy:: Youtube

IMPORTANT: Need to change Runtime Type to a GPU (at least A100) before running otherwise do not get a response when interacting with the chatbot.

Installation of Llama and Packages
Llama index is for creating the RAG framework
Fastembed is the embedding model, need to create the embeddings from your data to make it numerical for integrating with the LLM
Gradio is for the quick chatbot prototypes

Dependency Conflicts:
 - Transformers 4.40.1 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible
 - fastembed 0.2.6 requires tokenizers<0.16.0,>=0.15.1, but you have tokenizers 0.19.1 which is incompatible
 - Need to find the version of Transformers that uses the tokenizers <0.16.0, >=0.15.1; use transformers==4.39.2 (needs >= 0.14)

In [1]:
!pip install tokenizers==0.15.2
!pip install transformers==4.39.2
#!pip install -q pypdf!pip install -q python-dotenv!pip install llama-index==0.10.12!pip install -q gradio!pip install einops!pip install accelerate
!pip install pypdf
!pip install python-dotenv
!pip install llama-index==0.10.12
!pip install -q gradio
!pip install einops
!pip install accelerate
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-fastembed
!pip install fastembed
#!pip install llama-index==0.10.12 #DUPLICATE
#!pip install -q gradio #DUPLICATE

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.44.2 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 73.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 21.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-

Notes: completed above, there were grammatical errors that I needed to correct in the commented line where xxx!pip install should have been xxx <new line> !pip install. Also I could not find that the option -q exists for pip install so that was removed.
Lastly the tokenizer dependency of transformers and fastembed differ and are incompatible with versions 4.40.1 and 0.2.6 respectively, see above for the necessary transformer==4.39.2 version

Next section:
Logging and Data Loading
- Need to load the PDF file to the file directory each time, b/c information is purged from the system when no longer running to save on resources for others to use

In [15]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
documents = SimpleDirectoryReader("/content/Test_Bot").load_data() #The path here is under the directories of colab, changed to Test_Bot

Embedding Model and Settings

In [3]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
embed_model = FastEmbedEmbedding(model_name= "BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 512

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

Prompt and HuggingFace Integration

In [7]:
from llama_index.core import PromptTemplate
system_prompt = "You are an assistant to a physician. Your goal is to answer questions about medical guidelines as accurately as possible based on the instructions provided."
#FINISH / Update the prompt LATER
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")
from huggingface_hub import notebook_login
notebook_login()

Language Model Configuration

In [8]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
stopping_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>"),]
llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)
Settings.llm = llm
Settings.chunk_size = 512

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Building the Index and Query Engine

Note: upload the documents from which to build the query engine into the content/Test_Bot folder

In [16]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

Prediction Function

In [17]:
def predict(input, history):
  response = query_engine.query(input)
  return str(response)

Chatbot

In [18]:
import gradio as gr
gr.ChatInterface(predict).launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
